<a href="https://colab.research.google.com/github/NykPol/dw_matrix_road_sign/blob/master/day5_hyperparam_tun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from skimage import color,exposure
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score

%load_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test ['features'], test ['labels']

df = pd.read_csv('data/signnames.csv')
lables_dict = df.to_dict()['b']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
def train_model(model,X_train,y_train,params_fit={}):
  model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='Adam')

  logdir=os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callbacks = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
  model.fit(
      X_train
      ,y_train
      ,batch_size=params_fit.get('batch_size',128)
      ,epochs = params_fit.get('epochs',5)
      ,verbose = params_fit.get('verbose',1)
      ,validation_data=params_fit.get('validation_data',(X_train,y_train))
      ,callbacks = [tensorboard_callbacks]
      )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test,axis=1)

  y_pred_probs = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_probs,axis=1)

  return scoring(y_test_norm,y_pred)

In [0]:
def get_cnn_v6(input_shape,num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu',padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),


  Flatten(),

  Dense(1024,activation='relu'),
  Dropout(0.3),

  Dense(1024,activation='relu'),
  Dropout(0.3),

  
  Dense(num_classes, activation='softmax'),
])

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu',padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu',padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),


  Flatten(),

  Dense(1024,activation='relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024,activation='relu'),
  Dropout(params['dropout_dense_block_two']),

  
  Dense(num_classes, activation='softmax'),
])

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train,y_train,params_fit)
  return predict(model_trained, X_test, y_test)  

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [12]:
train_and_predict(get_cnn_v6(input_shape,num_classes)) 

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.5886 - accuracy: 0.2851 - val_loss: 0.8993 - val_accuracy: 0.7210
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.9005 - accuracy: 0.7154 - val_loss: 0.3081 - val_accuracy: 0.9082
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.4546 - accuracy: 0.8563 - val_loss: 0.0958 - val_accuracy: 0.9729
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.3065 - accuracy: 0.9074 - val_loss: 0.1001 - val_accuracy: 0.9697
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2281 - accuracy: 0.9322 - val_loss: 0.0638 - val_accuracy: 0.9797


0.9464852607709751

In [0]:
def func_obj(params):

  model = get_model(params)
  model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='Adam')

  model.fit(
      X_train
      ,y_train
      ,batch_size=int(params.get('batch_size',128))
      ,epochs = 5
      ,verbose = 0
      )
  
  accuracy = model.evaluate(X_test,y_test)[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss':-accuracy, 'status':STATUS_OK, 'model':model}


In [33]:
space = {
    'batch_size':hp.quniform('batch_size',100,200,10),
    'dropout_cnn_block_one':hp.uniform('dropout_cnn_block_one',0.3,0.5),
    'dropout_cnn_block_two':hp.uniform('dropout_cnn_block_two',0.3,0.5),
    'dropout_cnn_block_three':hp.uniform('dropout_cnn_block_three',0.3,0.5),

    'dropout_dense_block_one':hp.uniform('dropout_dense_block_one',0.3,0.7),
    'dropout_dense_block_two':hp.uniform('dropout_dense_block_two',0.3,0.7),
}
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 1.0231 - accuracy: 0.6250

 16/138 [==>...........................]
 - ETA: 0s - loss: 0.3474 - accuracy: 0.9023

 29/138 [=====>........................]
 - ETA: 0s - loss: 0.4742 - accuracy: 0.8793

 42/138 [========>.....................]
 - ETA: 0s - loss: 0.3491 - accuracy: 0.9129

 55/138 [==========>...................]
 - ETA: 0s - loss: 0.3813 - accuracy: 0.8898

 69/138 [==============>...............]
 - ETA: 0s - loss: 0.3437 - accuracy: 0.9022


Three Factors Influencing Minima in SGD
https://arxiv.org/abs/1711.04623